In [2]:
!%pip install --upgrade langchain
!%pip install --upgrade langchain-openai
!%pip install -qU langchain-core langchain-openai
!%pip install --upgrade openai
!%pip install azure-search
%pip install -r ../../../requirements.txt

/bin/bash: line 1: fg: no job control


/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control
/bin/bash: line 1: fg: no job control
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import json
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizedQuery, VectorQuery
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient

In [4]:

from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("This lab exercise will use the following values:")
    print("Azure OpenAI Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
    print("Azure AI Search: " + os.getenv("AZURE_AI_SEARCH_SERVICE_NAME"))
else: 
    print("No file .env found")

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("OPENAI_API_VERSION")
azure_openai_completion_deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
azure_openai_embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
azure_ai_search_name = os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")
azure_ai_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
azure_ai_search_index_name = os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
azure_ai_search_api_key = os.getenv("AZURE_AI_SEARCH_API_KEY")

This lab exercise will use the following values:
Azure OpenAI Endpoint: https://oai-mtpchatbot-dev-frcent-001.openai.azure.com/
Azure AI Search: srch-mtpchatbot-dev-frcent-001


In [5]:
import json
from openai import AzureOpenAI
import os

endpoint ="https://oai-mtpchatbot-dev-frcent-001.openai.azure.com/"
deployment='oaidepl-mtpchatbot-dev-frcent-003'

search_client = SearchClient(
    endpoint=azure_ai_search_endpoint, 
    index_name=azure_ai_search_index_name, 
    credential=azure_ai_search_api_key
)

client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    api_key="a74bc358eb354a7590d19d43fedc999a",
    api_version="2023-12-01-preview",
    )

rag_client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    api_key="a74bc358eb354a7590d19d43fedc999a",
    api_version="2023-12-01-preview",
    )



In [6]:
from langchain_openai import AzureOpenAIEmbeddings

azure_openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    openai_api_version = os.getenv("OPENAI_EMBEDDING_API_VERSION"),
    model= os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
)

from langchain_community.vectorstores.azuresearch import AzureSearch

index_name: str = "mtp-chatbot-index"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_AI_SEARCH_API_KEY"),
    index_name=index_name,
    embedding_function=azure_openai_embeddings.embed_query,
)

# Perform a similarity search testwise
results = vector_store.similarity_search(
    query="Wie schließe ich einen TruckLive Vertrag ab?",
    k=3, # forgot what this was -> k nearest neighbour?
    search_type="hybrid",
    # adjust relevance with score_threshold= xy,
)



print(results[0].page_content)

1

Schritt 1: 
Registrieren Sie sich für MyTruckPoint. 
Anschließend erscheint eine Schaltfläche zur
Registrierung von TruckLive. 
Um den Prozess zu beginnen, klicken Sie bitte
auf „Weiter“. (     )1

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



2

Schritt 2:
Stimmen Sie dem Daimler Rahmenvertrag
für digitale Dienste zu. (     )
Klicken Sie anschließend auf „Bestätigen
und Weiter“. (     )3

2

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



3

Schritt 3:
Stimmen Sie dem TruckLive Vertrag zu. (     )
Klicken Sie anschließend auf „Bestätigen
und Weiter“. (     )

4

5

TRUCKLIVE: KURZANLEITUNG VERTRAGSABSCHLUSS



4

Schritt 4:
Definieren Sie einen Zeitpunkt, ab dem TruckLive
für zukünftige Fahrzeuge aktiviert werden soll. (     )
Wählen Sie danach eine Kontaktperson aus dem
eigenen Unternehmen aus, welche die Meldungen 
zu TruckLive bekommen soll. (     ) 
Abschließend klicken Sie auf „Bestätigen“. (     )

Herzlichen Glückwunsch! 
TruckLive ist nun aktiviert und all Ihre Neufahrzeuge
e

In [7]:
def get_embeddings(text: str):
    # There are a few ways to get embeddings. This is just one example.
    import openai

    open_ai_endpoint = os.getenv(azure_openai_endpoint)
    open_ai_key = os.getenv(azure_openai_api_key)

    client = openai.AzureOpenAI(
        azure_endpoint=open_ai_endpoint,
        api_key=open_ai_key,
        api_version="2023-12-01-preview",

    )
    embedding = client.embeddings.create(input=[text], model=azure_openai_embedding_deployment_name)
    return embedding.data[0].embedding


def simple_hybrid_search(query):
    # [START simple_hybrid_search]

    search_client = SearchClient(azure_ai_search_endpoint, index_name, AzureKeyCredential(azure_ai_search_api_key))
    vector_query = VectorizedQuery(vector=get_embeddings(query), k_nearest_neighbors=3, fields="vector")

    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        select=["title", "chunk", "url"],
        top= 5,
    )

    n = 1
    results_for_prompt = ""
    for result in results:
        results_for_prompt += f"Result {result['title']}: {result['chunk']}: {result['url']}\n"
        n += 1
    
    return results_for_prompt
        
print(simple_hybrid_search("Was ist My TruckPoint?"))




Result versch. Wissensdokumente MTP aus Internet und Retail Panel.docx: über Ihre aktiven Serviceverträge und Informationen über den aktuellen Stand der jeweiligen Vertragslaufzeit und -laufleistung
Übersicht über Ihre gebuchten Fleetboard Dienste inklusive vereinfachter Fahrzeugaktivierung
Services & Teile für Ihren Lkw.
Flottentransparenz über anstehende Wartungen & Services
Fahrzeugspezifischer Service- & Teilebezug dank Lkw-Telematik in Kombination mit Shopfunktionalität
Ihr Weg zum neuen Lkw.
Bereitstellung Ihrer Fahrzeugangebote inklusive relevanter Dokumente
vollständige Übersicht Ihrer Bestellungen inklusive aktuellem Status


Auszug von DT Retail Portal 

Welche Funktionen bietet My TruckPoint dem Kunden?
	<p>In der Fahrzeugliste sieht der Kunde eine Übersicht über seinen Fuhrpark sowie aktive Verträge, z.B. Service- und Fleetboard Verträge. Die Daten rund ums Fahrzeug bzw. dem Vertrag werden sofern vorhanden automatisch zur Verfügung gestellt. </p><p>Darüber hinaus kann der K

In [12]:


# Function to generate embeddings for title and content fields, also used for query embeddings
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text):
    response = client.embeddings.create(input=text, model=azure_openai_embedding_deployment_name)
    embeddings = response.data[0].embedding
    return embeddings

def get_trucklive_status(vin: int):
    if vin == 123456789:
        return "acitvated"
    else:
        return "deactivated"

def get_current_contracts(user, password):
    if user == "yannik" and password == 123:
        return "TruckLive, Fleetboard, Uptime"
    else: return "None"

def search_sources(query):
    # [START simple_hybrid_search]

    search_client = SearchClient(azure_ai_search_endpoint, index_name, AzureKeyCredential(azure_ai_search_api_key))
    vector_query = VectorizedQuery(vector=get_embeddings(query), k_nearest_neighbors=3, fields="vector")

    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        select=["title", "chunk", "url"],
        top=5,
    )

    n = 1
    results_for_prompt = ""
    for result in results:
        results_for_prompt += f"Result {result['title']}: {result['chunk']}: {result['url']}\n"
        n += 1
    
    return results_for_prompt


def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = []
    messages.append({"role": "system", "content": """
    You're a polite AI assistant for MyTruckPoint. Do not make up anything.
    Give as much details as possible and be precise. Answer in the same language. If there are any steps, give them as lists. If it is a general question then reply appropriately
    
     """})
    messages.append({"role": "user", "content": "What is TruckLive?"})
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_trucklive_status",
                "description": "Returns the TruckLive Status of a vehicle given the VIN (Vehicle Identification Number).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "vin": {
                            "type": "integer",
                            "description": "the vehicle identification number",
                        },
                    },
                    "required": ["vin"],
                },
            }
        },
                {
            "type": "function",
            "function": {
                "name": "get_current_contracts",
                "description": "Return the current contracts the user is subscribed to given the user name and password",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "user": {
                            "type": "string",
                            "description": "the user name",
                        },
                        "password": {
                            "type": "string",
                            "description": "the password to the corresponding user name",
                        },
                    },
                    "required": ["user", "password"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "search_sources",
                "description": "Retrieve information about My TruckPoint, TruckLive and everything related from an Azure AI search index. Translate the results in the language of the given query paramter",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The query string to search for relevant data about My TruckPoint and everything",
                        },
                    },
                    "required": ["query"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_trucklive_status": get_trucklive_status,
            "get_current_contracts": get_current_contracts,
            "search_sources": search_sources
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
              # extend conversation with function response
        print(messages)
        second_response = client.chat.completions.create(
            model=deployment,
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response.model_dump_json(indent=2)
    else:
        return response.model_dump_json(indent=2)

print(run_conversation())


[{'role': 'system', 'content': "\n    You're a polite AI assistant for MyTruckPoint. Do not make up anything.\n    Give as much details as possible and be precise. Answer in the same language. If there are any steps, give them as lists. If it is a general question then reply appropriately\n    you should ONLY give emojis as answers.\n     "}, {'role': 'user', 'content': 'What is TruckLive?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_HKBdLPtpjSryiKRuVx6JRZXY', function=Function(arguments='{"query":"What is TruckLive?"}', name='search_sources'), type='function')]), {'tool_call_id': 'call_HKBdLPtpjSryiKRuVx6JRZXY', 'role': 'tool', 'name': 'search_sources', 'content': 'Result FAQ_MTP en-GB_041223.txt: for Digital Services & TruckLive Contract be downloaded?\tMy TruckPoint --> "Contract overview"\xa0\n\tTruckLive\tWhere can the contracts be viewed?\tMy TruckPoint --> "Contract overview"\xa0\n\tTruckLive\tWhe